In [2]:
import pandas as pd


In [4]:
df= pd.read_csv('fakenews_train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df=df.dropna()

In [6]:
## get independent features
x= df.drop('label',axis=1)

In [7]:
y=df['label']

In [8]:
x.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'1.13.1'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
# vocabularry size
voc_size=5000

# one hot representation

In [15]:
messages= x.copy()

In [16]:
messages.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Chaitanya
[nltk_data]     Kaul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [22]:
from tensorflow.keras.preprocessing.text import one_hot

In [23]:
onehot_repr= [one_hot(words,voc_size)for words in corpus]
print(onehot_repr)

[[426, 2288, 653, 4656, 2580, 3964, 81, 2417, 4127, 131], [551, 4887, 2029, 1288, 3529, 1458, 1002], [1757, 2841, 4757, 2494], [379, 4078, 3462, 2537, 2389, 1597], [4892, 3529, 4212, 973, 1651, 868, 3529, 1466, 2621, 27], [4152, 4075, 170, 528, 2573, 749, 2684, 1243, 2535, 2168, 3762, 2340, 272, 3940, 1002], [1834, 2426, 3704, 468, 2041, 2949, 3220, 19, 950, 1587, 4906], [3888, 2160, 2523, 1375, 3922, 4355, 749, 3895, 950, 1587, 4906], [1161, 2205, 3490, 1760, 3269, 4532, 3995, 3185, 749, 4922], [3669, 877, 3585, 3983, 2466, 705, 2410, 3341], [2460, 1221, 4010, 2224, 4160, 3663, 2135, 4525, 4216, 1971, 955], [2537, 3600, 2580, 4532, 749, 3922], [2171, 883, 4798, 3872, 239, 3776, 910, 1984, 1144], [2629, 1156, 2654, 697, 3123, 3692, 1398, 950, 1587, 4906], [1118, 4517, 3740, 1728, 151, 950, 1587, 4906], [1745, 167, 723, 2568, 2562, 4283, 763, 1922, 2459, 1966], [1754, 4745, 4887], [2732, 3000, 4314, 3440, 749, 1273, 1585, 1002], [432, 1505, 2029, 3019, 4900, 3690, 4508, 4692, 868], [480

# embedding representation

In [24]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2417 4127  131]
 [   0    0    0 ... 3529 1458 1002]
 [   0    0    0 ... 2841 4757 2494]
 ...
 [   0    0    0 ...  950 1587 4906]
 [   0    0    0 ... 4135 1355 3390]
 [   0    0    0 ... 4451 2956 1986]]


In [25]:
len(embedded_docs)

18285

In [37]:
## creating model
embedding_vector_features= 40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # making embedding layer
model.add(LSTM(100))  # one LSTM Layer with 100 neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
import numpy as np
x_final= np.array(embedded_docs)
y_final= np.array(y)

In [30]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=0)

# model training

In [31]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
12250/12250 [==============================] - 11s 897us/sample - loss: 0.3397 - acc: 0.8392 - val_loss: 0.1869 - val_acc: 0.9196
Epoch 2/10
12250/12250 [==============================] - 8s 672us/sample - loss: 0.1468 - acc: 0.9419 - val_loss: 0.1846 - val_acc: 0.9253
Epoch 3/10
12250/12250 [==============================] - 8s 690us/sample - loss: 0.1040 - acc: 0.9609 - val_loss: 0.2147 - val_acc: 0.9203
Epoch 4/10
12250/12250 [==============================] - 9s 711us/sample - loss: 0.0795 - acc: 0.9722 - val_loss: 0.2366 - val_acc: 0.9215
Epoch 5/10
12250/12250 [==============================] - 8s 686us/sample - loss: 0.0518 - acc: 0.9833 - val_loss: 0.2882 - val_acc: 0.9170
Epoch 6/10
12250/12250 [==============================] - 9s 696us/sample - loss: 0.0327 - acc: 0.9900 - val_loss: 0.3413 - val_acc: 0.9100
Epoch 7/10
12250/12250 [==============================] - 8s 6

In [32]:
y_pred= model.predict_classes(x_test)

In [36]:
y_pred

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [34]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm= confusion_matrix(y_test,y_pred)
print(cm)

[[3101  288]
 [ 244 2402]]


In [35]:
ac= accuracy_score(y_test,y_pred)
print(ac)

0.911847555923778


# adding dropout 

In [38]:
from tensorflow.keras.layers import Dropout


In [39]:
## creating model
embedding_vector_features= 40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # making embedding layer
model.add(Dropout(0.3))
model.add(LSTM(100))  # one LSTM Layer with 100 neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
import numpy as np
x_final= np.array(embedded_docs)
y_final= np.array(y)

In [41]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=0)

In [42]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 11s 902us/sample - loss: 0.3514 - acc: 0.8280 - val_loss: 0.1916 - val_acc: 0.9155
Epoch 2/10
12250/12250 [==============================] - 9s 736us/sample - loss: 0.1571 - acc: 0.9396 - val_loss: 0.1939 - val_acc: 0.9190
Epoch 3/10
12250/12250 [==============================] - 9s 727us/sample - loss: 0.1154 - acc: 0.9576 - val_loss: 0.1960 - val_acc: 0.9211
Epoch 4/10
12250/12250 [==============================] - 9s 748us/sample - loss: 0.0869 - acc: 0.9692 - val_loss: 0.2129 - val_acc: 0.9221
Epoch 5/10
12250/12250 [==============================] - 10s 778us/sample - loss: 0.0618 - acc: 0.9786 - val_loss: 0.2219 - val_acc: 0.9213
Epoch 6/10
12250/12250 [==============================] - 9s 711us/sample - loss: 0.0485 - acc: 0.9839 - val_loss: 0.2765 - val_acc: 0.9198
Epoch 7/10
12250/12250 [==============================] - 9s 736us/sample - loss: 0.0352 - acc: 0.9881 - val_

In [43]:
y_pred= model.predict_classes(x_test)

In [44]:
from sklearn.metrics import confusion_matrix, accuracy_score
cmm= confusion_matrix(y_test,y_pred)
print(cmm)

[[3074  315]
 [ 199 2447]]


In [45]:
ac= accuracy_score(y_test,y_pred)
print(ac)

0.9148301574150787
